# 📚 Task 2: Text Classification (Without Fine-Tuning)

## Model: `distilbert-base-uncased-finetuned-sst-2-english`

---

### 🎯 Goals
| Goal | Description |
|------|-------------|
| ✅ Feed custom sentences | Pass diverse inputs to the pipeline |
| ✅ Observe predictions | See POSITIVE / NEGATIVE labels |
| ✅ Analyse confidence scores | Understand per-label probabilities |
| ✅ Understand inference pipeline | End-to-end walkthrough |

---

### 🧠 Background

**DistilBERT** is a *distilled* (compressed) version of BERT, trained using knowledge distillation.
It preserves **97% of BERT's accuracy** while being **60% smaller** and **2× faster**.

The checkpoint `distilbert-base-uncased-finetuned-sst-2-english` was **fine-tuned on the SST-2 dataset**
(Stanford Sentiment Treebank), a binary sentiment classification task (POSITIVE / NEGATIVE).

> 💡 **No Fine-Tuning Required** — We are using the model exactly as it was released.
   This demonstrates *zero-shot domain transfer* for in-distribution sentiment tasks.

## ⚙️ Step 1 — Setup & Imports

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

import warnings
warnings.filterwarnings('ignore')

import torch
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from IPython.display import display, HTML
from transformers import pipeline

print(f'PyTorch version : {torch.__version__}')
print(f'CUDA available  : {torch.cuda.is_available()}')
print(f'Device          : {"GPU" if torch.cuda.is_available() else "CPU"}')

## 🔧 Step 2 — Load the Inference Pipeline

HuggingFace's `pipeline()` abstracts the full inference stack:

```
raw text
   ↓  Tokenizer      (WordPiece tokenisation + input IDs)
   ↓  DistilBERT     (contextual embeddings via 6 Transformer layers)
   ↓  Classifier Head (linear layer → logits)
   ↓  Softmax        (logits → probabilities)
   ↓  Label Decode   (argmax → POSITIVE / NEGATIVE)
predicted label + confidence score
```

In [ ]:
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'

print(f'Loading model: {MODEL_NAME} ...')

classifier = pipeline(
    task='text-classification',
    model=MODEL_NAME,
    device=-1,              # -1 = CPU; set to 0 for GPU
    return_all_scores=True  # return probabilities for ALL labels
)

print('✅ Model loaded successfully!')

## 💬 Step 3 — Define Custom Sentences

We design sentences across **5 categories** to probe different model behaviours:

| Category | Expected Behaviour |
|---|---|
| Clear positive | Very high confidence POSITIVE |
| Clear negative | Very high confidence NEGATIVE |
| Ambiguous | Low confidence, uncertain prediction |
| Negation traps | Tests contextual understanding |
| Mixed sentiment | Model must pick dominant signal |

In [ ]:
sentence_groups = {
    'Clear Positive': [
        'This movie is absolutely fantastic — I loved every second of it!',
        'The customer service was outstanding and the staff were incredibly kind.',
        'I have never tasted such an amazing meal in my entire life.',
    ],
    'Clear Negative': [
        'This product is a complete waste of money and I deeply regret buying it.',
        'Terrible experience. The room was dirty, cold, and the staff were rude.',
        'I hated this book. The plot was boring and the characters were flat.',
    ],
    'Ambiguous / Borderline': [
        'The film was okay — not great, but not terrible either.',
        'It is what it is. Some parts were decent, others felt forced.',
    ],
    'Negation Traps': [
        'Not bad at all! Actually quite impressive for the price.',
        'I would not say this was a bad experience, but it was not memorable.',
    ],
    'Mixed Sentiment': [
        'The food was delicious but the service was appallingly slow.',
        'Beautiful location, however the hotel rooms left much to be desired.',
    ],
}

# Flatten for batch inference
all_sentences = [s for sentences in sentence_groups.values() for s in sentences]
all_groups    = [g for g, sentences in sentence_groups.items() for _ in sentences]

print(f'Total sentences: {len(all_sentences)}')
for group, sentences in sentence_groups.items():
    print(f'  {group}: {len(sentences)} sentences')

## 🚀 Step 4 — Run Inference

In [ ]:
raw_outputs = classifier(all_sentences)

# Structure results into a clean list of dicts
results = []
for sentence, group, label_scores in zip(all_sentences, all_groups, raw_outputs):
    scores_dict = {ls['label']: ls['score'] for ls in label_scores}
    top = max(label_scores, key=lambda x: x['score'])
    results.append({
        'Group'     : group,
        'Sentence'  : sentence,
        'Label'     : top['label'],
        'Confidence': round(top['score'] * 100, 2),
        'POSITIVE'  : round(scores_dict.get('POSITIVE', 0) * 100, 2),
        'NEGATIVE'  : round(scores_dict.get('NEGATIVE', 0) * 100, 2),
    })

df = pd.DataFrame(results)
print(f'Inference complete for {len(df)} sentences.')
df

## 📊 Step 5 — Visualise Confidence Scores

### 5a. Per-sentence horizontal bar chart

In [ ]:
fig, ax = plt.subplots(figsize=(13, 7))

colours = ['#2ecc71' if lbl == 'POSITIVE' else '#e74c3c' for lbl in df['Label']]
short_labels = [s[:55] + '...' if len(s) > 55 else s for s in df['Sentence']]

bars = ax.barh(short_labels, df['Confidence'], color=colours, edgecolor='white', height=0.65)

# Annotate bars
for bar, conf, lbl in zip(bars, df['Confidence'], df['Label']):
    ax.text(
        min(conf + 1, 102), bar.get_y() + bar.get_height() / 2,
        f'{conf:.1f}%  {lbl}',
        va='center', ha='left', fontsize=9, fontweight='bold',
        color='#2ecc71' if lbl == 'POSITIVE' else '#e74c3c'
    )

ax.set_xlim(0, 115)
ax.set_xlabel('Confidence (%)', fontsize=12)
ax.set_title('Text Classification Confidence Scores\n(distilbert-base-uncased-finetuned-sst-2-english)',
             fontsize=14, fontweight='bold')
ax.axvline(80, color='orange', linestyle='--', alpha=0.7, label='80% threshold')
ax.axvline(99, color='gray',   linestyle='--', alpha=0.7, label='99% threshold')

pos_patch = mpatches.Patch(color='#2ecc71', label='POSITIVE')
neg_patch = mpatches.Patch(color='#e74c3c', label='NEGATIVE')
ax.legend(handles=[pos_patch, neg_patch], loc='lower right')

plt.tight_layout()
plt.savefig('../data/task2_confidence_bars.png', dpi=150)
plt.show()
print('Chart saved → data/task2_confidence_bars.png')

### 5b. Grouped box plot — confidence by sentence group

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))

group_order = list(sentence_groups.keys())
palette = {
    'Clear Positive'     : '#27ae60',
    'Clear Negative'     : '#c0392b',
    'Ambiguous / Borderline': '#f39c12',
    'Negation Traps'     : '#8e44ad',
    'Mixed Sentiment'    : '#2980b9',
}

sns.stripplot(
    data=df, x='Group', y='Confidence', hue='Label',
    palette={'POSITIVE': '#2ecc71', 'NEGATIVE': '#e74c3c'},
    order=group_order, size=12, jitter=0.15, ax=ax
)

ax.axhline(80, color='orange', linestyle='--', alpha=0.8, label='80% threshold')
ax.axhline(99, color='gray',   linestyle='--', alpha=0.8, label='99% threshold')
ax.set_xlabel('Sentence Group', fontsize=11)
ax.set_ylabel('Confidence (%)', fontsize=11)
ax.set_title('Confidence Scores by Sentence Group', fontsize=13, fontweight='bold')
ax.tick_params(axis='x', rotation=15)
ax.legend(loc='lower left')

plt.tight_layout()
plt.savefig('../data/task2_group_confidence.png', dpi=150)
plt.show()
print('Chart saved → data/task2_group_confidence.png')

### 5c. POSITIVE vs NEGATIVE score scatter (diverging plot)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

colours_scatter = ['#2ecc71' if lbl == 'POSITIVE' else '#e74c3c' for lbl in df['Label']]

ax.scatter(df['POSITIVE'], df['NEGATIVE'], c=colours_scatter, s=120, edgecolors='white', linewidths=1.5, zorder=3)

# Diagonal line — where POSITIVE = NEGATIVE = 50%
ax.plot([0, 100], [100, 0], 'k--', alpha=0.3, label='Decision boundary (50/50)')

# Annotate each point
for _, row in df.iterrows():
    short = row['Sentence'][:30] + '...'
    ax.annotate(short, (row['POSITIVE'], row['NEGATIVE']),
                fontsize=6.5, ha='left', va='bottom',
                xytext=(3, 3), textcoords='offset points', alpha=0.8)

ax.set_xlabel('POSITIVE score (%)', fontsize=12)
ax.set_ylabel('NEGATIVE score (%)', fontsize=12)
ax.set_title('POSITIVE vs NEGATIVE Confidence Scores\n(points near the diagonal = uncertain predictions)',
             fontsize=12, fontweight='bold')

pos_patch = mpatches.Patch(color='#2ecc71', label='Predicted POSITIVE')
neg_patch = mpatches.Patch(color='#e74c3c', label='Predicted NEGATIVE')
ax.legend(handles=[pos_patch, neg_patch, plt.Line2D([0], [0], color='k', linestyle='--', alpha=0.3, label='Decision boundary')])

plt.tight_layout()
plt.savefig('../data/task2_pos_vs_neg.png', dpi=150)
plt.show()
print('Chart saved → data/task2_pos_vs_neg.png')

## 📈 Step 6 — Analysis Summary

In [ ]:
total     = len(df)
n_pos     = (df['Label'] == 'POSITIVE').sum()
n_neg     = (df['Label'] == 'NEGATIVE').sum()
high_conf = (df['Confidence'] >= 99).sum()
low_conf  = (df['Confidence'] <  80).sum()

print('=' * 55)
print('  ANALYSIS SUMMARY')
print('=' * 55)
print(f'  Total sentences       : {total}')
print(f'  POSITIVE predictions  : {n_pos} ({n_pos/total*100:.1f}%)')
print(f'  NEGATIVE predictions  : {n_neg} ({n_neg/total*100:.1f}%)')
print(f'  Avg confidence        : {df["Confidence"].mean():.2f}%')
print(f'  High-conf (≥ 99%)     : {high_conf} sentences')
print(f'  Low-conf  (< 80%)     : {low_conf}  sentences')
print('=' * 55)

print('\n  Confidence by group:')
group_summary = df.groupby('Group')['Confidence'].agg(['mean', 'min', 'max']).round(2)
group_summary.columns = ['Mean %', 'Min %', 'Max %']
display(group_summary)

## 🔬 Step 7 — Deep Dive: Tokenisation Walkthrough

Let's look at **how the tokenizer converts raw text** before passing it to the model.

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

demo_sentences = [
    'Not bad at all! Actually quite impressive for the price.',
    'The food was delicious but the service was appallingly slow.',
]

print('=' * 65)
print('  TOKENISATION WALKTHROUGH')
print('=' * 65)

for sent in demo_sentences:
    tokens   = tokenizer.tokenize(sent)
    token_ids = tokenizer.encode(sent)
    print(f'\n  Input   : {sent}')
    print(f'  Tokens  : {tokens}')
    print(f'  IDs     : {token_ids}')
    print(f'  # tokens: {len(tokens)} (+ 2 special tokens [CLS]/[SEP] = {len(token_ids)})')

## 🧪 Step 8 — Interactive: Try Your Own Sentence

In [ ]:
def classify_and_display(text: str) -> None:
    """Classify a single sentence and display a formatted result."""
    output = classifier([text])[0]  # list of label scores
    scores = {ls['label']: ls['score'] for ls in output}
    top    = max(output, key=lambda x: x['score'])
    
    bar_pos = '█' * int(scores['POSITIVE'] * 30) + '░' * (30 - int(scores['POSITIVE'] * 30))
    bar_neg = '█' * int(scores['NEGATIVE'] * 30) + '░' * (30 - int(scores['NEGATIVE'] * 30))
    
    print(f'\n  Input     : "{text}"')
    print(f'  Prediction: {top["label"]}  ({top["score"]*100:.2f}% confident)')
    print(f'  POSITIVE  : [{bar_pos}] {scores["POSITIVE"]*100:.2f}%')
    print(f'  NEGATIVE  : [{bar_neg}] {scores["NEGATIVE"]*100:.2f}%')


# ── Try your own sentences here! ──────────────────────────────────────────────
classify_and_display('I am so excited about this new project!')
classify_and_display('The weather is cloudy today.')
classify_and_display('This was the worst decision I have ever made in my career.')

## 📝 Step 9 — Key Takeaways

| Concept | What We Learned |
|---|---|
| **Inference Pipeline** | `pipeline()` chains tokenization → model → softmax in one call |
| **DistilBERT** | 66M params, 6 Transformer layers, fast and accurate for NLP |
| **Confidence Scores** | Softmax probabilities summing to 1.0 across labels |
| **High Confidence** | Clear sentiment sentences score ≥ 99% |
| **Low Confidence** | Ambiguous / mixed sentences score 50–80% |
| **Negation** | Model handles 'not bad' correctly via contextual attention |
| **No Fine-Tuning** | Pre-trained checkpoint works out-of-the-box for reviews/sentiment |
| **Tokenisation** | WordPiece splits words; [CLS] and [SEP] special tokens are added |

---

### 🚀 Next Steps
- **Task 3**: Named Entity Recognition (NER) with `dslim/bert-base-NER`
- **Task 4**: Question Answering — extractive QA with `deepset/roberta-base-squad2`
- **Fine-Tuning**: Adapt the model to your own domain-specific sentiment data